## Problem Description

A post office requires different numbers of full-time employees on different days of the week. The number of full-time employees required on each day is given in Table below. Union rules state that each full-time employee must work five consecutive days and then receive two days off. For example, an employee who works Monday to Friday must be off on Saturday and Sunday. The post office wants to meet its daily requirements using only fulltime employees. Formulate an LP that the post office can use to minimize the number of full-time employees who must be hired.

**Reference:**

*Winston, Wayne L., and Jeffrey B. Goldberg. Operations research: applications and algorithms, 4th edition, 2004, Pg. 60*

*This problem was solved as part of "Section 4:Introduction to linear programming" in the course Mathematical Optimization with GAMS and Pyomo (Python) on udemy. (https://www.udemy.com/course/mathematical-optimization-with-gams-and-pyomo-python/)*

## Data

| Day | No. of Employees required |
| :- | :--: |
| 1 = Monday | 17 |
| 2 = Tuesday | 13 |
| 3 = Wednesday | 15 |
| 4 = Thursday | 19 |
| 5 = Friday | 14 |
| 6 = Saturday | 16 |
| 7 = Sunday | 11 |


## Solution

**Sets:**

$t \in \text{Days} = \{\text{1},\text{2},\text{3},\text{4},\text{5},\text{6},\text{7}\}$

**Parameters:**

$\text{demand}_{t} \in \mathbb{I}^+$: Employees required on day $t$.

$\theta_{t} \subset \text{Days}$: Subset of start days from $\text{Days}$ when a worker ends up working on day $t$.


**Variables:**

$\text{x}_{t} \in \mathbb{I}^+$: No. of employees that start working from day $t$.

**Objective Function:**

\begin{equation}
\text{Min} \sum_{t \in Days} \text{x}_{t}
\tag{0}
\end{equation}

**Constraints:**

\begin{equation}
\sum_{i \in \theta_{t}} \text{x}_{i} \ge \text{demand}_{t} \quad \forall t \in \text{Days}
\tag{1}
\end{equation}









In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyomo

     |████████████████████████████████| 9.2 MB 15.2 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

In [ ]:
# Defining Model
model = pyomo.ConcreteModel()

# Defining Sets
model.dow = pyomo.RangeSet(7, doc = 'Day of the week, from monday')

# Defining Variables
model.x = pyomo.Var(model.dow, domain = pyomo.NonNegativeIntegers)

# Declaring Parameters
model.demand = pyomo.Param(model.dow, initialize = {1:17, 2:13, 3:15, 4:19, 5:14, 6:16, 7:11})

dict_ = {}
arr = list(range(1,8))
for i in arr:
  temp_arr = arr[i-1:7]+arr[0:i-1]
  del temp_arr[1:3]
  dict_[i] = temp_arr
model.theta = pyomo.Param(model.dow, initialize = dict_)

# Defining Obj
model.obj = pyomo.Objective(expr = sum(model.x[a] for a in model.dow), sense = pyomo.minimize)

# Defining Constraints
def rule1(model, a):
  return sum(model.x[d] for d in model.theta[a]) >= model.demand[a]
model.equ1 = pyomo.Constraint(model.dow, rule = rule1, doc = 'Min. worker requirement')

# Solver options
results = pyomo.SolverFactory('glpk', executable = '/usr/bin/glpsol').solve(model)

# Print results
results.write()
print('\n Results \n')
print('Total no. of employees:', model.obj())
print('No. of employees starting monday:', model.x[1]())
print('No. of employees starting tuesday:', model.x[2]())
print('No. of employees starting wednesday:', model.x[3]())
print('No. of employees starting thrusday:', model.x[4]())
print('No. of employees starting friday:', model.x[5]())
print('No. of employees starting saturday:', model.x[6]())
print('No. of employees starting sunday:', model.x[7]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 23.0
  Upper bound: 23.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 8
  Number of nonzeros: 36
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 17
      Number of created subproblems: 17
  Error rc: 0
  Time: 0.012519598007202148
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------